# [공공데이터 포털 - 기상청_단기예보 ((구)_동네예보) 조회서비스 Open API]를 이용하여 날씨 안내 AI 고객센터 챗봇 만들기 - Few-shot examples 추가

## Reference : https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15084084

# 초단기실황조회 API (getUltraSrtNcst) 정의

In [ ]:
from datetime import datetime
import requests
import json

In [ ]:
def get_current_date():
    current_date = datetime.now().date()
    return current_date.strftime("%Y%m%d")

def get_current_hour():
    now = datetime.now()
    return datetime.now().strftime("%H%M")

In [ ]:
weather_enum = {
    "0": "맑음",
    "1": "비",
    "2": "비/눈",
    "3": "눈",
    "5": "빗방울",
    "6": "빗방울눈날림",
    "7": "눈날림"
}

In [ ]:
def parse_weather_infomation(result):
    for item in result['response']['body']['items']['item']:
        if item['category'] == 'PTY':
            weather = weather_enum[item['obsrValue']]
        if item['category'] == 'REH':
            습도 = item['obsrValue']
        if item['category'] == 'RN1':
            한시간강수량 = item['obsrValue']
        if item['category'] == 'T1H':
            기온 = item['obsrValue']

    return_string = ''
    return_string += f'날씨 : {weather}\n'
    return_string += f'습도 : {습도}\n'
    return_string += f'1시간강수량 : {한시간강수량}\n'
    return_string += f'기온 : {기온}\n'

    return return_string

# 예보지점 X 좌표, 예보지점 Y 좌표 얻기

*   참고문서 - [기상청41_단기예보 조회서비스_오픈API활용가이드_최종.zip] 파일 다운로드후 압축풀기

In [ ]:
import pandas as pd

In [ ]:
file_path = '/content/기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(20240101).xlsx'

In [ ]:
# 데이터 프레임으로 읽어오기
df = pd.read_excel(file_path)
df

In [ ]:
df.columns

# 날씨 안내 AI 고객센터 챗봇 만들기

In [ ]:
!pip install --upgrade --quiet langchain langchainhub langchain-openai

### Decoding 키 환경변수 설정

In [ ]:
import os
os.environ["decoding_일반_인증키"] = '여러분의_Decoding_일반_인증키'

### LangSmith 설정

In [ ]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Weather API Chatbot (Add Few shot examples) - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "여러분의_LANGSMITH_API_KEY"

In [ ]:
unique_id

###  OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "여러분의_OPENAI_API_KEY"

### LLM 설정

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", openai_api_key=OPENAI_KEY)

### Tool 정의

*   df를 전역변수에서 가져오도록 함수 로직 수정


In [ ]:
# 예보지점 X 좌표, 예보지점 Y를 얻어오는 함수 정의
def get_grid_coordinates(level1, level2=None, level3=None):
    query = f"`1단계` == '{level1}'"
    if level2 is not None:
        query += f" and `2단계` == '{level2}'"
    if level3 is not None:
        query += f" and `3단계` == '{level3}'"

    result = df.query(query)
    if not result.empty:
        grid_x = result.iloc[0]['격자 X']
        grid_y = result.iloc[0]['격자 Y']
        return grid_x, grid_y
    else:
        return None, None

# level1 지명 보정 함수 작성

In [ ]:
def level1_correction(level1):
    if level1 in ['서울시', '서울']:
        level1 = '서울특별시'
    if level1 in ['부산시', '부산']:
        level1 = '부산광역시'
    if level1 in ['대구시', '대구']:
        level1 = '대구광역시'
    if level1 in ['인천시', '인천']:
        level1 = '인천광역시'
    if level1 in ['광주시', '광주']:
        level1 = '광주광역시'
    if level1 in ['대전시', '대전']:
        level1 = '대전광역시'
    if level1 in ['울산시', '울산']:
        level1 = '울산광역시'
    if level1 in ['세종시', '세종']:
        level1 = '세종특별자치시'
    if level1 in ['제주도', '제주']:
        level1 = '제주특별자치도'
    if level1 in ['강원도', '강원']:
        level1 = '강원특별자치도'

    return level1

In [ ]:
level1_correction('서울')

In [ ]:
from langchain_core.tools import tool

@tool
def get_weather_infomation(level1: str, level2: str = None, level3: str = None) -> str:
    """Get weather information."""
    x_position, y_position = get_grid_coordinates(level1_correction(level1), level2, level3)

    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst' # 초단기실황조회
    params = {
              'serviceKey' : os.environ.get('decoding_일반_인증키'),
              'pageNo' : '1',
              'numOfRows' : '1000',
              'dataType' : 'JSON',
              'base_date' : get_current_date(),
              'base_time' : get_current_hour(),
              'nx' : x_position,
              'ny' : y_position
             }

    response = requests.get(url, params=params)
    weather_info = parse_weather_infomation(json.loads(response.text))

    return weather_info

In [ ]:
print(get_weather_infomation.name)
print(get_weather_infomation.description)
print(get_weather_infomation.args)

In [ ]:
get_weather_infomation.invoke({"level1": '서울특별시'})

In [ ]:
get_weather_infomation.invoke({"level1": '부산'})

### tool agent 설정

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [ ]:
# Get the prompt to use - can be replaced with any prompt that includes variables "agent_scratchpad" and "input"!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

In [ ]:
prompt

In [ ]:
tools = [get_weather_infomation]

In [ ]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Few-shot examples 추가

In [ ]:
# Using with chat history
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.messages import AIMessage

chat_history = [
    HumanMessage(
        "서울특별시 종로구 청운효자동의 현재 날씨를 알려줘", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "get_weather_infomation", "args": {"level1": "서울특별시", "level2": "종로구", "level3": "청운효자동"}, "id": "1"}
        ],
    ),
    ToolMessage("날씨 : 맑음\n습도 : 83\n1시간강수량 : 0\n기온 : 23.6\n", tool_call_id="1"),
    HumanMessage(
        "제주도에 현재 비가 오는 중이야?", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "get_weather_infomation", "args": {"level1": "제주도"}, "id": "2"}
        ],
    ),
    ToolMessage("날씨 : 맑음\n습도 : 81\n1시간강수량 : 0\n기온 : 21.2\n", tool_call_id="2"),
]

In [ ]:
msg_result = agent_executor.invoke(
    {
        "chat_history": chat_history,
        "input": "서울특별시의 영등포구 여의동의 날씨를 알려줘"
    }
)
msg_result

In [ ]:
msg_result['output']

In [ ]:
msg_result = agent_executor.invoke(
    {
        "chat_history": chat_history,
        "input": "대구의 날씨를 알려줘"
    }
)
msg_result

In [ ]:
msg_result['output']

In [ ]:
msg_result = agent_executor.invoke(
    {
        "chat_history": chat_history,
        "input": "강원도 원주시 행구동에 현재 비가 오는지 알려줘"
    }
)
msg_result

In [ ]:
msg_result['output']